In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

In [ ]:
data = pd.read_csv('/kaggle/input/country-socioeconomic-data/Country-data.csv')
data.head(10)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
print(data.shape)

In [ ]:
df = data.iloc[:, 1:data.shape[1]]
df

In [ ]:
# checking the missing values
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
# checking for outliers using box-plots

plt.figure(figsize=(20,20), dpi=100)

plt.subplot(4,3,1)
sns.boxplot(x = 'child_mort', data = data)

plt.subplot(4,3,2)
sns.boxplot(x = 'exports', data = data)

plt.subplot(4,3,3)
sns.boxplot(x = 'health', data = data)

plt.subplot(4,3,4)
sns.boxplot(x = 'imports', data = data)

plt.subplot(4,3,5)
sns.boxplot(x = 'income', data = data)

plt.subplot(4,3,6)
sns.boxplot(x = 'inflation', data = data)

plt.subplot(4,3,7)
sns.boxplot(x = 'life_expec', data = data)

plt.subplot(4,3,8)
sns.boxplot(x = 'total_fer', data = data)

plt.subplot(4,3,9)
sns.boxplot(x = 'gdpp', data = data)

In [ ]:
# checking for outliers using the Z-score

from scipy import stats

z = np.abs(stats.zscore(data[['child_mort', 'exports', 'health', 'imports', 'income',
       'inflation', 'life_expec', 'total_fer', 'gdpp']]))
print(z)
print("*******************************************************************************")

# threshold = 3  # selecting 3 as the threshold to identify outliers
print('Below are the outlier points along with the respective column numbers in the second array')
print("\n")
print(np.where(z > 3))

In [ ]:
data_outliers_removed = data[(z<3).all(axis=1)]

In [ ]:
data_outliers_removed.head()

In [ ]:
print('Shape of dataframe before outlier removal: ' + str(data.shape))
print("\n")
print('Shape of dataframe after outlier removal: ' + str(data_outliers_removed.shape))

In [ ]:
num_vars = data_outliers_removed.drop('country',axis =1)  
cat_vars = data_outliers_removed['country']

In [ ]:
num_vars

**PCA**

In [ ]:
# Standardization of the dataset before performing PCA
scaler = StandardScaler()
num_vars_scaled = scaler.fit_transform(num_vars)
num_vars_scaled_df = pd.DataFrame(num_vars_scaled,columns=num_vars.columns)
num_vars_scaled_df

In [ ]:
# Let's see the correlation matrix 
plt.figure(figsize = (20,10))       
sns.heatmap(num_vars_scaled_df.corr(),annot = True)

we can see that some columns have significant correlation among themselves 

In [ ]:
pca = PCA(random_state=42)
pca.fit(num_vars_scaled)
pca

In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)
fig = plt.figure(figsize=[12,8],dpi=80)
plt.vlines(x=4, ymax=1, ymin=0, colors="r", linestyles="--")
plt.hlines(y=0.95, xmax=30, xmin=0, colors="g", linestyles="--")
plt.plot(var_cumu)
plt.ylabel("Cumulative variance explained")
plt.show()

**Performing PCA with 4 components**

In [ ]:
pca_final = IncrementalPCA(n_components=4)
num_vars_pca_final = pca_final.fit_transform(num_vars_scaled)
print(num_vars.shape)
print(num_vars_pca_final.shape)

# Clustering

**K - means clustering**

In [ ]:
# Finding the Optimal Number of Clusters
ssd = []
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=1000)
    kmeans.fit(num_vars_pca_final)
    
    ssd.append(kmeans.inertia_)
    
# plot the SSDs for each n_clusters
# ssd
plt.plot(ssd)

 from the elbow method we can say that k = 4 clusters seems to be a good choice

In [ ]:
# silhouette analysis
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:
    
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=1000)
    kmeans.fit(num_vars_pca_final)
    
    cluster_labels = kmeans.labels_
    
    # silhouette score
    silhouette_avg = silhouette_score(num_vars_pca_final, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

In [ ]:
# final model with k=3
kmeans = KMeans(n_clusters=4, max_iter=1000, random_state=42)
kmeans.fit(num_vars_pca_final)

In [ ]:
data_outliers_removed['K-Means_Cluster_ID'] = kmeans.labels_

# DBSCAN Clustering

In [ ]:
import sklearn.utils

In [ ]:
healthy_df_clus_temp = df[["child_mort","income","gdpp",]]
healthy_df_clus_temp = StandardScaler().fit_transform(healthy_df_clus_temp)
db = DBSCAN(eps = 0.3, min_samples = 3).fit(healthy_df_clus_temp)
labels = db.labels_
labels

In [ ]:
dataFrameLabels = pd.DataFrame(labels)
#dataFrameLabels

In [ ]:
# Building the label to colour mapping 
colours = {} 
colours[0]  = 'red' # Zero values on labels
colours[1]  = 'green'# Minus 1 values on labels
colours[-1]  = 'blue'# Two values on labels

In [ ]:
cvec = [colours[label] for label in labels]
colors = ['red', 'green', 'b'] 
print(cvec)
set(cvec)

In [ ]:
#Visualize Results
# For the construction of the legend of the plot 
colors = ['r','g','b']
r = plt.scatter( df["child_mort"],df["income"], marker ='o', color =colours[0] ) 
g = plt.scatter( df["child_mort"],df["gdpp"], marker ='o', color = colours[1]) 
b = plt.scatter( df["income"],df["gdpp"], marker ='o', color = colours[-1]) 
plt.figure(figsize =(9, 9)) 
plt.scatter(df["health"],df["life_expec"], c = cvec) 
plt.legend((r, g, b), 
           ('Label 0', 'Label 2', 'Label -1'), 
           scatterpoints = 1, 
           loc ='upper right', 
           ncol = 3, 
           fontsize = 8) 

# Hierarchical Clustering


In [ ]:
num_vars_scaled_df.head()

In [ ]:
# complete linkage
cl_mergings = linkage(num_vars_scaled_df, method="complete", metric='euclidean')
dendrogram(cl_mergings)
plt.show()

In [ ]:
# 4 clusters using complete linkage
cl_cluster_labels = cut_tree(cl_mergings, n_clusters=4).reshape(-1, )
cl_cluster_labels

In [ ]:
data_outliers_removed["Hierarchical_Cluster_labels"] = cl_cluster_labels


In [ ]:
data_outliers_removed.head()


As we can see, all rows are assigned to one cluster

In [ ]:
# plotting sub-plots to analyse the results

plt.figure(figsize=(18,15), dpi=100)

plt.subplot(3,2,1)
sns.boxplot(x='K-Means_Cluster_ID', y='gdpp', data=data_outliers_removed)

plt.subplot(3,2,2)
sns.boxplot(x='Hierarchical_Cluster_labels', y='gdpp', data=data_outliers_removed)

plt.subplot(3,2,3)
sns.boxplot(x='K-Means_Cluster_ID', y='child_mort', data=data_outliers_removed)

plt.subplot(3,2,4)
sns.boxplot(x='Hierarchical_Cluster_labels', y='child_mort', data=data_outliers_removed)

plt.subplot(3,2,5)
sns.boxplot(x='K-Means_Cluster_ID', y='income', data=data_outliers_removed)

plt.subplot(3,2,6)
sns.boxplot(x='Hierarchical_Cluster_labels', y='income', data=data_outliers_removed)

Scatter plot using the gdpp, child_mortt, income to observe the cluster distribution

In [ ]:
# scatter plot using the gdpp, child_mort to observe the cluster distribution

plt.figure(figsize=(15,6),dpi=90)

plt.subplot(1,2,1)
sns.scatterplot(x='gdpp',y='child_mort',data=data_outliers_removed,hue='K-Means_Cluster_ID')

plt.subplot(1,2,2)
sns.scatterplot(x='gdpp',y='child_mort',data=data_outliers_removed,hue='Hierarchical_Cluster_labels')



In [ ]:
plt.figure(figsize=(15,6),dpi=90)

plt.subplot(1,2,1)
sns.scatterplot(x='gdpp',y='income',data=data_outliers_removed,hue='K-Means_Cluster_ID')

plt.subplot(1,2,2)
sns.scatterplot(x='gdpp',y='income',data=data_outliers_removed,hue='Hierarchical_Cluster_labels')

In [ ]:
plt.figure(figsize=(15,6),dpi=90)

plt.subplot(1,2,1)
sns.scatterplot(x='child_mort',y='income',data=data_outliers_removed,hue='K-Means_Cluster_ID')

plt.subplot(1,2,2)
sns.scatterplot(x='child_mort',y='income',data=data_outliers_removed,hue='Hierarchical_Cluster_labels')

# Countries which are in direst need of aid

From K-Means clustering,

We can see that countries with cluster labelled as '1' are in need of aid

In [ ]:
K_Means_countries = data_outliers_removed[data_outliers_removed['K-Means_Cluster_ID'] == 1]
K_Means_countries.head(10)

From Hierarchical clustering,

We can see that countries with cluster labelled as '0' are in need of aid

In [ ]:
Hirarchical_countries = data_outliers_removed[data_outliers_removed['Hierarchical_Cluster_labels'] == 0]
Hirarchical_countries.head(10)

In [ ]:
# countries common to both the models

common_countries = pd.merge(K_Means_countries,Hirarchical_countries,how='inner',on=['country', 'child_mort', 'exports', 'health', 'imports', 'income',
       'inflation', 'life_expec', 'total_fer', 'gdpp', 'K-Means_Cluster_ID',
       'Hierarchical_Cluster_labels'])

In [ ]:
common_countries[['country', 'child_mort', 'income','gdpp']].head(10)

In [ ]:
# dataframe with dereasing child mortality rate and increasing income

common_countries_final = common_countries[['country', 'child_mort','income','gdpp']].sort_values(['child_mort','income'],ascending=[False,True])
common_countries_final.head(15)

### Countries with direst need for aid can be selected from the above dataframe


In [ ]:
final_countries = common_countries_final[(common_countries_final['child_mort'] > 80) &  (common_countries_final['income'] < 1200)]
final_countries = final_countries.reset_index(drop=True)
final_countries

In [ ]:
import numpy as np 
from sklearn.cluster import DBSCAN 
from sklearn import metrics 
from sklearn.datasets.samples_generator import make_blobs 
from sklearn.preprocessing import StandardScaler 
from sklearn import datasets 
  
# Load data in X 
db = DBSCAN(eps=0.3, min_samples=10).fit(X) 
core_samples_mask = np.zeros_like(db.labels_, dtype=bool) 
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_ 
  
# Number of clusters in labels, ignoring noise if present. 
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0) 
  
print(labels) 
  
# Plot result 
import matplotlib.pyplot as plt 
  
# Black removed and is used for noise instead. 
unique_labels = set(labels) 
colors = ['y', 'b', 'g', 'r'] 
print(colors) 
for k, col in zip(unique_labels, colors): 
    if k == -1: 
        # Black used for noise. 
        col = 'k'
  
    class_member_mask = (labels == k) 
  
    xy = X[class_member_mask & core_samples_mask] 
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col, 
                                      markeredgecolor='k',  
                                      markersize=6) 
  
    xy = X[class_member_mask & ~core_samples_mask] 
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col, 
                                      markeredgecolor='k', 
                                      markersize=6) 
  
plt.title('number of clusters: %d' %n_clusters_) 
plt.show() 